In [1]:
import numpy as np

In [2]:
class Parameters():
    
    def __init__(self, parameter_values):
        self.parameters = parameter_values
        self.connections = set()
        
    def add_connections(self, conn):
        self.connections.add(conn)
    
    def get_parameters(self):
        return self.parameters
    
    def __repr__(self):
        return f'parameters for {self.connections}'

In [3]:
class NeuronConnection():
    
    def __init__(self, from_unit, to_unit, weight):
        self.from_unit = from_unit
        self.to_unit = to_unit
        self.weight = Parameters(weight)
        self.weight.add_connections(self)
    
    def __repr__(self):
        return f'{str(self.from_unit)} to {str(self.to_unit)}'
    

In [4]:
class Neuron():
    
    def __init__(self, name):
        self.name = name
        self.energy_potential = 0
        self.connections = []
        self.activation_function = None
    
    def update(self, energy_potential):
        self.energy_potential += energy_potential
    
    def fire(self):
        
        if self.activation_function != None:
            current_energy_potential = self.activation_function(self.energy_potential)
        else:
            current_energy_potential = self.energy_potential
        
        for conn in self.connections:
            conn.to_unit.update(current_energy_potential*conn.weight.get_parameters())
    
    def flush_energy(self):
        self.energy_potential = 0
        for conn in self.connections:
            conn.to_unit.flush_energy()
    
    def __repr__(self): # makes printing a neuron object reveal its name
        return self.name

In [5]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
sigmoid(-1000), sigmoid(0), sigmoid(1000)

/var/folders/hd/kdltsywn53jbp7lv3g1hld3w0000gn/T/ipykernel_50402/234028601.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))


(0.0, 0.5, 1.0)

In [6]:

x = [Neuron('x0'), Neuron('x1'), Neuron('x2')]
z1 = [Neuron('z1_0'), Neuron('z1_1'), Neuron('z1_2')]
z2 = [Neuron('z2_0'), Neuron('z2_1'), Neuron('z2_2')]
y = Neuron('y')
L = Neuron('L')


In [7]:
x[0].connections = [ # w^1_01, w^1_02
    NeuronConnection(
        from_unit=x[0],
        to_unit=z1[1],
        weight=-1
    ),
    NeuronConnection(
        from_unit=x[0],
        to_unit=z1[2],
        weight=1
    )
]
# x[0].activation_function = sigmoid

x[1].connections = [ # w^1_11, w^1_12
    NeuronConnection(
        from_unit=x[1],
        to_unit=z1[1],
        weight=-2
    ),
    NeuronConnection(
        from_unit=x[1],
        to_unit=z1[2],
        weight=2
    )
]
# x[1].activation_function = sigmoid

x[2].connections = [ # w^1_21, w^1_22
    NeuronConnection(
        from_unit=x[2],
        to_unit=z1[1],
        weight=-3
    ),
    NeuronConnection(
        from_unit=x[2],
        to_unit=z1[2],
        weight=3
    )
]
# x[2].activation_function = sigmoid

In [8]:
z1[0].connections = [ # w^2_01, w^2_02
    NeuronConnection(
        from_unit=z1[0],
        to_unit=z2[1],
        weight=-1
    ),
    NeuronConnection(
        from_unit=z1[0],
        to_unit=z2[2],
        weight=1
    )
]
# z1[0].activation_function = sigmoid

z1[1].connections = [ # w^2_11, w^2_12
    NeuronConnection(
        from_unit=z1[1],
        to_unit=z2[1],
        weight=-2
    ),
    NeuronConnection(
        from_unit=z1[1],
        to_unit=z2[2],
        weight=2
    )
]
z1[1].activation_function = sigmoid

z1[2].connections = [ # w^2_11, w^2_12
    NeuronConnection(
        from_unit=z1[2],
        to_unit=z2[1],
        weight=-3
    ),
    NeuronConnection(
        from_unit=z1[2],
        to_unit=z2[2],
        weight=3
    )
]
z1[2].activation_function = sigmoid

In [9]:
z2[0].connections = [ # w^3_01
    NeuronConnection(
        from_unit=z2[0],
        to_unit=y,
        weight=-1
    )
]
# z2[0].activation_function = sigmoid

z2[1].connections = [ # w^3_01
    NeuronConnection(
        from_unit=z2[1],
        to_unit=y,
        weight=2
    )
]
z2[1].activation_function = sigmoid

z2[2].connections = [ # w^3_01
    NeuronConnection(
        from_unit=z2[2],
        to_unit=y,
        weight=-1.5
    )
]
z2[2].activation_function = sigmoid

In [10]:
y.connections = [ #
    NeuronConnection(
        from_unit=y,
        to_unit=L,
        weight=1
    )
]

In [11]:
def run(X):
    
    x[0].flush_energy()
    x[1].flush_energy()
    x[2].flush_energy()
    
    x[0].energy_potential = 1
    z1[0].energy_potential = 1
    z2[0].energy_potential = 1
    
    x[1].energy_potential = X[0]
    x[2].energy_potential = X[1]
    
    x[0].fire()
    x[1].fire()
    x[2].fire()
    print(z1[0].energy_potential, z1[1].energy_potential, z1[2].energy_potential)
    
    z1[0].fire()
    z1[1].fire()
    z1[2].fire()
    print(z2[0].energy_potential, z2[1].energy_potential, z2[2].energy_potential)
    
    z2[0].fire()
    z2[1].fire()
    z2[2].fire()
    
    return y.energy_potential

In [12]:
run([1,1])

1 -6 6
1 -3.9975273768433657 3.9975273768433657


-2.436895226556018

# ^ matches problem 2

In [15]:
class Backprop:
    
    def __init__(self):
        self.__cache__ = {}
        
    def find_unit_paths(self, connection, from_node, to_node, current_path=None, visited=None):
        if current_path is None:
            current_path = []
        if visited is None:
            visited = set()

        if not connection is None:
            current_path.append(connection)
            visited.add(connection)

        if from_node == to_node:
            # Found a path
            yield list(current_path)
        else:
            for neighbor in from_node.connections:
                if neighbor not in visited:
                    yield from self.find_unit_paths(neighbor, neighbor.to_unit, to_node, current_path, visited)

        if not connection is None:
            current_path.pop()
            visited.remove(connection)

    def build_gradient(self, from_node, to_parameter):
        for parameter_connection in to_parameter.connections:
            to_node = parameter_connection.to_unit
            print(from_node, to_node)
            paths = list(self.find_unit_paths(None, to_node, from_node))
            assert len(paths)>0, 'No path between unit and parameter found.'
            print(to_parameter)
            print(paths)
            gradient = []
            for conns in paths:
                partial = 1
                for conn in conns:
                    partial*=conn.weight.get_parameters()
                gradient.append(partial)

In [17]:
n = Backprop().build_gradient(
    L,
    x[0].connections[0].weight
)

L z1_1
parameters for {x0 to z1_1}
[[z1_1 to z2_1, z2_1 to y, y to L], [z1_1 to z2_2, z2_2 to y, y to L]]


In [ ]:
z1[0].connections[0].weight.parent_units

In [ ]:
n

In [ ]:
for i in set([1, 2, 3]):
    print(i)